In [1]:
# if running for first time, you may need to install spacy model
# > python -m spacy download en_core_web_sm
import dotenv, pyreadr, os
dotenv.load_dotenv()

from pathlib import Path
from datetime import datetime
import pandas as pd
import sqlalchemy as sa
import sqlalchemy.orm as so

from hemonc_alchemy.matchers.spacy_config import matchers, nlp, rules, match_entities, get_modifier_child, get_nouns  
from hemonc_alchemy.model.hemonc_model import Hemonc_Study, Hemonc_Condition, Hemonc_Component, Hemonc_Modality, Hemonc_Component_Class, Hemonc_Regimen, Hemonc_Variant, Hemonc_Regimen_Part, Part_Phase, Hemonc_Cycle_Sig, Hemonc_Sig, Sig_Days, Base, component_to_class_map, Hemonc_Ref, Hemonc_Component_Role, Hemonc_Context, Hemonc_Branch_Conditional
from hemonc_alchemy.model.hemonc_enums import Intent, Setting, Phase, Risk, Phenotype, PriorTherapy, Phase, StudyDesign, SponsorType, BranchConditionalType, ComponentRole

In [10]:
# load data objects

# download the updated hemonc tables folder from dropbox and unzip it here
TABLE_PATH = Path('..') / 'HemOnc KB' / 'Tables'

# download the required athena vocab files and place them here
CDM_PATH = Path('..') / 'OHDSI_VOCABS' 

In [113]:
concept_columns = ['concept_name', 'domain_id', 'vocabulary_id', 'concept_class_id', 'standard_concept', 'concept_code','valid_start_date', 'valid_end_date', 'invalid_reason']
concept_relationship_columns = ['concept_code_1', 'concept_code_2', 'vocabulary_id_1', 'vocabulary_id_2', 'relationship_id', 'valid_start_date', 'valid_end_date', 'invalid_reason']

sigs = pd.read_csv(TABLE_PATH / 'sigs.csv', low_memory=False)
omop = pyreadr.read_r(TABLE_PATH / 'omop.RData')
concept_stage = omop['concept_stage'][concept_columns].copy()
concept_relationship_stage = omop['concept_relationship_stage'][concept_relationship_columns].copy()
concept_synonym_stage = omop['concept_synonym_stage'].copy()

# using RData files where possible, to avoid versioning issues
context = pyreadr.read_r(TABLE_PATH / 'context.table.RData')['context.table']
page = pyreadr.read_r(TABLE_PATH / 'page.table.RData')['page.table']
pointer = pyreadr.read_r(TABLE_PATH / 'pointer.table.RData')['pointer.table']
ref = pyreadr.read_r(TABLE_PATH / 'ref.table.RData')['ref.table']
sequence = pyreadr.read_r(TABLE_PATH / 'sequence.table.RData')['sequence.table']
study = pyreadr.read_r(TABLE_PATH / 'study.table.RData')['study.table']

In [5]:
author = pyreadr.read_r(TABLE_PATH / 'author.table.RData')['author.table']
person = pyreadr.read_r(TABLE_PATH / 'person.table.RData')['person.table']
variant = pyreadr.read_r(TABLE_PATH / 'variant.table.RData')['variant.table']

In [6]:
# following files had non-utf characters, requiring manual R-based conversion to csv because this is not handled with pyreadr at this time
# author = pd.read_csv(TABLE_PATH / 'author.table.csv', low_memory=False)
# person = pd.read_csv(TABLE_PATH / 'person.table.csv', encoding="ISO-8859-1")
# variant = pd.read_csv(TABLE_PATH / 'variant.table.csv', encoding="ISO-8859-1")

regimen = pointer[pointer.regimen_cui!='NOT YET ASSIGNED'][['regimen', 'regimen_cui']].drop_duplicates().copy()

In [11]:
CDM_PATH.resolve()

PosixPath('/Users/z3061723/Documents/CODE/HemOnc_Alchemy/OHDSI_VOCABS')

In [13]:
# load minimal athena download files for cross-reference including concept IDs where available
concepts_with_ids = pd.read_csv(CDM_PATH / 'CONCEPT.csv', delimiter='\t', low_memory=False)
concept_relationships = pd.read_csv(CDM_PATH / 'CONCEPT_RELATIONSHIP.csv', delimiter='\t', low_memory=False)
concept_ancestors = pd.read_csv(CDM_PATH / 'CONCEPT_ANCESTOR.csv', delimiter='\t', low_memory=False)
vocabularies = pd.read_csv(CDM_PATH / 'VOCABULARY.csv', delimiter='\t', low_memory=False)
concept_class = pd.read_csv(CDM_PATH / 'CONCEPT_CLASS.csv', delimiter='\t', low_memory=False)
domain = pd.read_csv(CDM_PATH / 'DOMAIN.csv', delimiter='\t', low_memory=False)
relationship = pd.read_csv(CDM_PATH / 'RELATIONSHIP.csv', delimiter='\t', low_memory=False)

In [14]:
import omop_alchemy as oa

engine = oa.oa_config.engine
oa.Base.metadata.create_all(engine)

In [15]:
concepts_with_ids.head()

,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_code,valid_start_date,valid_end_date,invalid_reason
0,36189414,hemorrhoidal cream 10mg/g / 144mg/g / 150mg/g ...,Drug,NDC,9-digit NDC,NaN,003630641,20180325,20991231,NaN
1,1220863,fulvestrant 250mg/5mL INTRAMUSCULAR INJECTION,Drug,NDC,9-digit NDC,NaN,167290436,20210121,20991231,NaN
2,3541502,Adverse reaction to drug primarily affecting t...,Condition,SNOMED,Disorder,NaN,694331000000106,20100401,20111003,D
3,3542356,Adverse reaction to other central nervous syst...,Condition,SNOMED,Disorder,NaN,705311000000105,20100401,20111003,D
4,42538812,Somatic hallucination,Condition,SNOMED,Clinical Finding,S,762620006,20180131,20991231,NaN


In [16]:
# at this stage, these are the classes that exist in HemOnc but aren't pulled into OMOP

non_omop_class = ['Author', 'Reference', 'Study', 'PubMedURL', 'ReferenceDOI', 'Clinical trial ID', 'Regimen Variant', 
                  'ReferenceURL', 'PubMedCentralURL', 'City', 'Cycle Sigs', 'Regimen Stub', 'Study Group', 'Journal', 
                  'Endpoint', 'Duration', 'Year', 'Numeric', 'Endpoint Type', 'Experimental design', 'Study Class']

cc = concept_stage[~concept_stage.concept_class_id.isin(non_omop_class)].merge(concepts_with_ids, on='concept_code', how='left')

In [17]:
# how many of the HemOnc concepts that are in OMOP classes do not currently have an assigned concept_id?

len(cc[cc.concept_id.isna()])/len(cc)

0.2846381948941682

In [18]:
# current hemonc concepts from which classes are missing IDs from Athena?

cc[cc.concept_id.isna()].concept_class_id_x.value_counts()

concept_class_id_x
PMID               6147
PMCID              2011
Regimen             129
Regimen Class        19
Unit                 12
Component            10
Brand Name            7
Frequency             6
Component Role        3
Condition             3
Context               3
Component Class       1
Name: count, dtype: int64

In [19]:
# trying to clean up NDC concept codes but it doesn't really work

concept_relationship_stage.concept_code_2 = (
    concept_relationship_stage.apply(
        lambda x: x.concept_code_2.replace('-', '') if x.vocabulary_id_2 == "NDC" else x.concept_code_2, 
        axis=1
        )
)

cr = (
    concept_relationship_stage[concept_relationship_stage.vocabulary_id_2 != 'HemOnc']
    .merge(concepts_with_ids, 
           left_on=['concept_code_2', 'vocabulary_id_2'], 
           right_on=['concept_code', 'vocabulary_id'], 
           how='left')
)

In [20]:
# which non-hemonc vocabs do we need to have available for allowing this to actually extend the CDM?
concept_relationship_stage.vocabulary_id_2.value_counts()

vocabulary_id_2
HemOnc              434438
RxNorm               20007
NDC                  14588
ICD-O-3               1618
ICD-10-CM             1430
ICD-9-CM              1021
NCIT                   738
HCPCS                  513
RxNorm Extension       483
SEER Site Recode       166
OncoTree               145
Name: count, dtype: int64

In [21]:
concept_relationship_stage[concept_relationship_stage.vocabulary_id_2=='NDC']

,concept_code_1,concept_code_2,vocabulary_id_1,vocabulary_id_2,relationship_id,valid_start_date,valid_end_date,invalid_reason
rownames,,,,,,,,
1100235,4,00024483,HemOnc,NDC,Maps to,2022-02-05,2099-12-31,
285916,4,00024815,HemOnc,NDC,Maps to,2022-02-05,2099-12-31,
310209,4,00025337,HemOnc,NDC,Maps to,2022-02-05,2099-12-31,
410201,4,00026216,HemOnc,NDC,Maps to,2022-02-05,2099-12-31,
510173,6,00931125,HemOnc,NDC,Maps to,2022-02-05,2099-12-31,
...,...,...,...,...,...,...,...,...
1296910,93818,50242103,HemOnc,NDC,Maps to,2022-02-05,2099-12-31,
1297010,93818,50242105,HemOnc,NDC,Maps to,2022-02-05,2099-12-31,
1396710,94609,25682022,HemOnc,NDC,Maps to,2022-02-05,2099-12-31,


In [22]:
concepts_with_ids.vocabulary_id.value_counts()

vocabulary_id
RxNorm Extension        2146945
NDC                     1254857
SNOMED                  1089088
RxNorm                   311332
LOINC                    274904
OSM                      203339
ICDO3                     64471
NAACCR                    34473
ICD10                     16638
HemOnc                    13376
HCPCS                     12306
Cancer Modifier            6043
OMOP Extension             1459
UCUM                       1128
CDM                        1061
Relationship                744
Concept Class               433
UB04 Typ bill               298
SOPT                        168
Vocabulary                  150
Condition Type              118
Procedure Type               97
Type Concept                 80
Domain                       65
UB04 Pt dis status           55
Cost                         51
Observation Type             29
UB04 Point of Origin         23
Condition Status             22
Visit                        20
Visit Type                

In [23]:
# not sure how much this matters, but there seems to be some issues with hemonc mappings to NDC - codes are a bit of a mish-mash with dropped hyphens and leading 0s removed
cr[cr.concept_id.isna()].vocabulary_id_2.value_counts()

vocabulary_id_2
NDC                 10615
ICD-O-3              1618
ICD-10-CM            1430
ICD-9-CM             1021
NCIT                  738
SEER Site Recode      166
OncoTree              145
HCPCS                  14
RxNorm                 10
Name: count, dtype: int64

In [24]:
code_mods = concept_stage.merge(
    concepts_with_ids[
        ['concept_code', 'concept_id', 'vocabulary_id', 'concept_name']
    ], 
    on=['concept_code', 'vocabulary_id'],
    how='left'
)

In [25]:
# there are some hemonc components that have changed concept_code? this will not be backwards compatible?
code_mods[(code_mods.concept_name_x != code_mods.concept_name_y) & ~code_mods.concept_class_id.isin(non_omop_class) & code_mods.concept_name_y.notna()].to_csv('reused_codes.csv', index=False)

In [26]:
vocab_id_lookup = {
    'ICD-10-CM': 'ICD10CM',
    'ICD-9-CM': 'ICD9CM',
    'ICD-O-3': 'ICDO3'
}

concept_relationship_stage['vocabulary_id'] = concept_relationship_stage.vocabulary_id_2.map(lambda x: vocab_id_lookup[x] if x in vocab_id_lookup else x)

In [27]:
# get all the OMOP concept IDs that exist for the current OMOP-ready hemonc codes

concept_stage_id = concept_stage.merge(
    concepts_with_ids[
        ['concept_code', 'concept_id', 'vocabulary_id', 'concept_name'] # note that merging on concept_name shouldn't be required here, but as per above with code mods, some of the concept codes in hemonc have changed over time
    ], 
    how='left'
)

# also for the hemonc concepts in the relationship file

concept_relationship_stage_id = concept_relationship_stage.merge(
    concepts_with_ids[
        ['concept_code', 'concept_id', 'vocabulary_id']
    ].rename(
        columns={'concept_code': 'concept_code_1', 
                 'concept_id': 'concept_id_1', 
                 'vocabulary_id': 'vocabulary_id_1'}
        ), 
    how='left'
)

# and then the related concepts in the relationship file

concept_relationship_stage_id = concept_relationship_stage_id.merge(
    concepts_with_ids[
        ['concept_code', 'concept_id', 'vocabulary_id']
    ].rename(
        columns={'concept_code': 'concept_code_2', 
                 'concept_id': 'concept_id_2'}
        ), 
    how='left'
)

In [28]:
# we need to grab all the concepts specifically for vocab, relationships, domains and concept classes so we can insert those as bulk initial load and avoid referential integrity issues

vocab_concepts = vocabularies.merge(
    concepts_with_ids[[c for c in concepts_with_ids.columns if c != 'vocabulary_id']], 
    left_on='vocabulary_concept_id', 
    right_on='concept_id'
)

rel_concepts = relationship.merge(
    concepts_with_ids[[c for c in concepts_with_ids.columns if c != 'relationship_id']], 
    left_on='relationship_concept_id', 
    right_on='concept_id'
)

domain_concepts = domain.merge(
    concepts_with_ids[[c for c in concepts_with_ids.columns if c != 'domain_id']], 
    left_on='domain_concept_id', 
    right_on='concept_id'
)

concept_class_concepts = concept_class.merge(
    concepts_with_ids[[c for c in concepts_with_ids.columns if c != 'concept_class_id']], 
    left_on='concept_class_concept_id', 
    right_on='concept_id'
)

In [29]:
# this is where we are building in the compatibility with the OMOP Alchemy classes

from omop_alchemy.model.vocabulary import Concept, Vocabulary, Domain, Concept_Class, Relationship, Concept_Relationship, Concept_Ancestor

def make_concept(concept_row):
    return Concept(concept_id = int(concept_row.concept_id),
                   concept_name = concept_row.concept_name if not pd.isna(concept_row.concept_name) else "",
                   domain_id = concept_row.domain_id,
                   vocabulary_id = concept_row.vocabulary_id,
                   concept_class_id = concept_row.concept_class_id,
                   standard_concept = concept_row.standard_concept if not pd.isna(concept_row.standard_concept) else None,
                   concept_code = concept_row.concept_code,
                   valid_start_date = datetime.strptime(str(concept_row.valid_start_date), '%Y%m%d'), 
                   valid_end_date = datetime.strptime(str(concept_row.valid_end_date), '%Y%m%d'), 
                   invalid_reason = concept_row.invalid_reason if not pd.isna(concept_row.invalid_reason) else None)

def make_vocab(vocab_row):
    v = Vocabulary(vocabulary_id = vocab_row.vocabulary_id,
                   vocabulary_name = vocab_row.vocabulary_name,
                   vocabulary_reference = vocab_row.vocabulary_reference,
                   vocabulary_concept_id = vocab_row.vocabulary_concept_id, 
                   vocabulary_version = vocab_row.vocabulary_version)
    v.vocabulary_concept = vocab_row.concept_object
    return v
    
def make_domain(domain_row):
    d = Domain(domain_id = domain_row.domain_id,
               domain_name = domain_row.domain_name,
               domain_concept_id = domain_row.domain_concept_id)
    d.domain_concept = domain_row.concept_object
    return d

def make_concept_class(cc_row):
    c = Concept_Class(concept_class_id = cc_row.concept_class_id,
                      concept_class_name = cc_row.concept_class_name,
                      concept_class_concept_id = cc_row.concept_class_concept_id)
    c.concept_class_concept = cc_row.concept_object
    return c


def make_rel(r_row):
    r = Relationship(relationship_id = r_row.relationship_id,
                      relationship_name = r_row.relationship_name,
                      is_hierarchical = r_row.is_hierarchical,
                      defines_ancestry = r_row.defines_ancestry,
                      reverse_relationship_id = r_row.reverse_relationship_id,
                      relationship_concept_id = r_row.relationship_concept_id)
    r.relationship_concept = r_row.concept_object
    return r

In [30]:
# preserve referential integrity by adding all the concept class, vocabulary and domain concepts in one go
meta_concepts = pd.concat([concept_class_concepts.concept_id, domain_concepts.concept_id, vocab_concepts.concept_id, rel_concepts.concept_id])
meta_concepts_df = concepts_with_ids.merge(meta_concepts)
meta_concepts_df['concept_object'] = meta_concepts_df.apply(make_concept, axis=1)


concept_class_concepts = concept_class_concepts.merge(meta_concepts_df[['concept_id', 'concept_object']], left_on='concept_class_concept_id', right_on='concept_id')
domain_concepts = domain_concepts.merge(meta_concepts_df[['concept_id', 'concept_object']], left_on='domain_concept_id', right_on='concept_id')
vocab_concepts = vocab_concepts.merge(meta_concepts_df[['concept_id', 'concept_object']], left_on='vocabulary_concept_id', right_on='concept_id')
rel_concepts = rel_concepts.merge(meta_concepts_df[['concept_id', 'concept_object']], left_on='relationship_concept_id', right_on='concept_id')

In [31]:
# full set of hemonc relationships - some obviously do not need to be pulled in - TBC how to cut down

concept_relationship_stage.relationship_id.value_counts()

relationship_id
Has middle author       115337
Is a                     67508
Was studied in           38112
Maps to                  21025
Has modality             12568
                         ...  
Has anticoag Rx             21
Has radiotherapy Rx          9
Was KFDA approved yr         5
Has pept-drug cjgt           1
Has PDC Rx                   1
Name: count, Length: 90, dtype: int64

In [32]:
# if you're pulling in only hemonc concepts that have a valid domain_id, then you can retain only relationships that have concepts in allocated domains on both sides
# this is kind of circular logic, but an ok place to start

concept_stage_id.domain_id.value_counts()

domain_id
               98499
regimen         8330
drug            6337
measurement      823
condition        287
procedure        115
Name: count, dtype: int64

In [33]:
domain_lookup_by_code = {k:v for k, v in zip(concept_stage_id.concept_code, concept_stage_id.domain_id)}
domain_lookup_by_id = {k:v for k, v in zip(concepts_with_ids.concept_id, concepts_with_ids.domain_id)}

In [34]:
# try find all known domain_id values either from hemonc source or existing OMOP 

concept_rels_with_domains = concept_relationship_stage_id[concept_relationship_stage_id.invalid_reason != 'D'].copy()
concept_rels_with_domains['domain_id_1'] = concept_rels_with_domains.concept_code_1.map(domain_lookup_by_code)
concept_rels_with_domains['domain_id_2'] = concept_rels_with_domains.concept_code_2.map(domain_lookup_by_code)
concept_rels_with_domains.loc[concept_rels_with_domains['domain_id_1'].isna(), 'domain_id_1'] = concept_rels_with_domains.concept_id_1.map(domain_lookup_by_id)
concept_rels_with_domains.loc[concept_rels_with_domains['domain_id_2'].isna(), 'domain_id_2'] = concept_rels_with_domains.concept_id_2.map(domain_lookup_by_id)

In [35]:
# that would imply that these are the relationship types that you need to exist before import
concept_rels_with_domains[
    (concept_rels_with_domains.domain_id_1!='') & 
    (concept_rels_with_domains.domain_id_2!='') 
    ].relationship_id.value_counts()

relationship_id
Is a                    64873
Maps to                 20670
Has cytotoxic chemo     10907
Has context              7524
Is current in adult      6883
                        ...  
Has anticoag Rx            14
Has radiotherapy Rx         8
Was KFDA approved yr        5
Has pept-drug cjgt          1
Has PDC Rx                  1
Name: count, Length: 64, dtype: int64

In [36]:
target_rels = concept_rels_with_domains[(concept_rels_with_domains.domain_id_1!='') & (concept_rels_with_domains.domain_id_2!='') ].relationship_id.value_counts().reset_index()
target_rels = target_rels.merge(relationship, how='left')

In [37]:
# means these may be the new relationship types being sought - confirmation required with respect to the 'year' concept class?
target_rels[target_rels.relationship_name.isna()]

,relationship_id,count,relationship_name,is_hierarchical,defines_ancestry,reverse_relationship_id,relationship_concept_id
4,Is current in adult,6883,NaN,NaN,NaN,NaN,NaN
8,Has been compared to,4373,NaN,NaN,NaN,NaN,NaN
10,Has synthetic regimen,3059,NaN,NaN,NaN,NaN,NaN
14,Can be preceded by,1100,NaN,NaN,NaN,NaN,NaN
16,Can be followed by,1005,NaN,NaN,NaN,NaN,NaN
19,Is historical in adult,710,NaN,NaN,NaN,NaN,NaN
21,Was FDA approved yr,565,NaN,NaN,NaN,NaN,NaN
23,Has major class,430,NaN,NaN,NaN,NaN,NaN
26,Has minor class,381,NaN,NaN,NaN,NaN,NaN
30,Was EMA approved yr,308,NaN,NaN,NaN,NaN,NaN


In [38]:
# query why one of the vocabularies has a null ID from athena?

vocab_concepts[vocab_concepts.vocabulary_id.isna()]

,vocabulary_id,vocabulary_name,vocabulary_reference,vocabulary_version,vocabulary_concept_id,concept_id_x,concept_name,domain_id,concept_class_id,standard_concept,concept_code,valid_start_date,valid_end_date,invalid_reason,concept_id_y,concept_object
13,NaN,OMOP Standardized Vocabularies,OMOP generated,v5.0 27-FEB-25,44819096,44819096,OMOP Standardized Vocabularies,Metadata,Vocabulary,NaN,OMOP generated,19700101,20991231,NaN,44819096,<Concept 44819096 - OMOP generated (OMOP Stand...


In [39]:
vocab_concepts.vocabulary_id = vocab_concepts.vocabulary_id.fillna('empty') 

In [40]:
def make_ancestor(a_row):
    return Concept_Ancestor(ancestor_concept_id = int(a_row.ancestor_concept_id),
                            descendant_concept_id = int(a_row.descendant_concept_id))
                            #min_levels_of_separation = a_row.min_levels_of_separation,
                            #max_levels_of_separation = a_row.max_levels_of_separation)
                    

In [41]:
# create metadata concepts & make objects for other reference tables
concept_class_concepts['ob'] = concept_class_concepts.apply(make_concept_class, axis=1)
domain_concepts['ob'] = domain_concepts.apply(make_domain, axis=1)
vocab_concepts['ob'] = vocab_concepts.apply(make_vocab, axis=1)
rel_concepts['ob'] = rel_concepts.apply(make_rel, axis=1)

all_ob = pd.concat([concept_class_concepts[['concept_id_x', 'ob']], 
                    domain_concepts[['concept_id_x', 'ob']], 
                    vocab_concepts[['concept_id_x', 'ob']], 
                    rel_concepts[['concept_id_x', 'ob']]])

In [43]:

with so.Session(engine) as sess:
    # sess.add_all(list(concept_class_concepts.ob))
    # sess.add_all(list(domain_concepts.ob))
    # sess.add_all(list(vocab_concepts.ob))
    # sess.add_all(list(rel_concepts.ob))
    # sess.add_all(list(meta_concepts_df.concept_object))
    # sess.commit()

SyntaxError: incomplete input (1956843125.py, line 7)

In [48]:
with so.Session(engine) as sess:
    existing_vocabs = pd.DataFrame(sess.query(Concept.vocabulary_id).distinct().all())
    existing_concepts = pd.DataFrame(sess.query(Concept.concept_id).distinct().all())

existing_vocabs.head()

,vocabulary_id
0,SNOMED
1,CDM
2,Cancer Modifier
3,Concept Class
4,Condition Status


In [45]:
concept_stage_id[~concept_stage_id.concept_class_id.isin(non_omop_class) & 
                 ~concept_stage_id.vocabulary_id.isin(['Concept Class', 'Domain', 'Vocabulary', 'Relationship'])  
#                 &~concept_stage_id.concept_id.isin(existing_concepts.concept_id.unique())
                 ].concept_class_id.value_counts()

concept_class_id
PMID               8299
Regimen            5970
Brand Name         4981
PMCID              2457
Component           837
Regimen Class       795
Component Class     505
Condition           287
Procedure            64
Context              51
Unit                 37
Modality             23
Route                14
Frequency            14
Component Role        3
Null                  1
Name: count, dtype: int64

In [46]:
concepts_to_add = concept_relationship_stage_id[
    ['concept_code_2', 'vocabulary_id']
    ].drop_duplicates().merge(
        concepts_with_ids, 
        left_on=['concept_code_2', 'vocabulary_id'], 
        right_on=['concept_code', 'vocabulary_id']
    )

In [49]:
len(concepts_to_add[~concepts_to_add.concept_id.isin(existing_concepts.concept_id.unique())])

0

In [50]:
# add in concepts both from within the HemOnc universe, as well as all available cross-mappings
ho_conc = concepts_with_ids[concepts_with_ids.vocabulary_id=='HemOnc'].apply(make_concept, axis=1)
non_ho_conc = concepts_to_add[~concepts_to_add.vocabulary_id.isin(['Concept Class', 'Domain', 'Vocabulary', 'Relationship', 'HemOnc'])].apply(make_concept, axis=1)

In [ ]:
with so.Session(engine) as sess:
    sess.add_all(ho_conc)
    sess.add_all(non_ho_conc)
    sess.commit()

In [52]:
# # uncomment this section only if you want to pull in extended vocab, not just those pointed to by hemonc relationships

# # use this one if you want completeness for vocabs that are involved in hemonc relationships, even the concepts that aren't pointed 
# # to directly by those relationships
# # target_vocabularies = concept_relationship_stage.vocabulary_id.unique()

# # use this one if you want useful oncology-extension relevant vocabs
# target_vocabularies = ['Cancer Modifier', 'OMOP Extension', 'NAACCR', 'Gender', 'Episode Type', 
#                        'Visit', 'Condition Type', 'Procedure Type', 'Type Concept', 'Observation Type',
#                        'Visit Type', 'Drug Type', 'Death Type', 'Meas Type', 'Note Type',
#                        'Cost Type', 'Obs Period Type', 'Episode Type', 'Device Type', 'Domain', 
#                        'Ethnicity', 'Language', 'Relationship', 'Race', 'CDM', 'Concept Class',
#                        'RxNorm', 'ICD10CM', 'ICD03', 'Cancer Modifier']

# # you can handle a larger chunksize if using non-sqlite backend - 50k is large but reasonable on a local device if sqlite

# chunksize = 50000

# for vocab in target_vocabularies:
#     v = concepts_with_ids[concepts_with_ids.vocabulary_id==vocab]
#     if len(v) > 0:
#         with so.Session(engine) as sess:
#             existing = pd.DataFrame(sess.query(Concept.concept_id).filter(Concept.vocabulary_id==vocab))
#             if len(existing) > 0:
#                 v = v.merge(existing, how="left", indicator=True)
#                 v = v[v._merge=='left_only']
#             for i in range(0, len(v), chunksize):
#                 to_add_objects = v[i:i+chunksize].apply(make_concept, axis=1)
#                 sess.add_all(to_add_objects)
#                 sess.commit()
#     print(vocab, len(v), len(existing), len(v))



In [ ]:
with so.Session(engine) as sess:
    existing_vocabs = pd.DataFrame(sess.query(Concept.vocabulary_id).distinct().all())
    existing_concepts = pd.DataFrame(sess.query(Concept.concept_id).distinct().all())

existing_vocabs.head()

In [ ]:
def make_relationship(rel_row):
    return Concept_Relationship(concept_id_1 = rel_row.concept_id_1,
                         concept_id_2 = rel_row.concept_id_2,
                         relationship_id = rel_row.relationship_id)

# we cannot set relationships for any new concepts that do not have concept_id assigned, so note that these will be missing

ho_rels = concept_relationship_stage_id[
    concept_relationship_stage_id.concept_id_1.notna() &
    concept_relationship_stage_id.concept_id_2.notna()
    ].drop_duplicates(
        subset=['concept_id_1', 'concept_id_2', 'relationship_id']
        ).apply(make_relationship, axis=1)

In [ ]:
ancestors = concept_ancestors.merge(
    existing_concepts[
        ['concept_id']
        ].rename(columns={'concept_id': 'ancestor_concept_id'})
    ).merge(
        existing_concepts[
            ['concept_id']
        ].rename(columns={'concept_id': 'descendant_concept_id'})
    ).apply(make_ancestor, axis=1)  

In [ ]:
len(ancestors), len(ho_rels)

In [ ]:
with so.Session(engine) as sess:
    for i in range(0, len(ancestors), 50000):
        sess.add_all(list(ancestors[i:i+50000]))
        sess.commit()

In [ ]:
with so.Session(engine) as sess:
    sess.add_all(ho_rels)
    sess.commit()

In [ ]:
# uncomment this section if you want to add extended relationships that involve only non-hemonc concepts for specific extra vocabs
# don't need to do the same for ancestry, as we don't filter that by vocab - it's done by existence of concepts already above, so if you pulled
# them in, the ancestry will be there

chunksize = 50000

# get only relationships where both concepts have been imported

target_relationships = (
    concept_relationships
    .merge(existing_concepts.rename(columns={'concept_id': 'concept_id_1'}))
    .merge(existing_concepts.rename(columns={'concept_id': 'concept_id_2'}))
)

with so.Session(engine) as sess:
    existing_relationships = pd.DataFrame(
        sess.query(
            Concept_Relationship.concept_id_1, Concept_Relationship.concept_id_2, Concept_Relationship.relationship_id
            )
    )

relationships_to_add = target_relationships.merge(existing_relationships, how='left', indicator=True)
relationships_to_add = relationships_to_add[relationships_to_add._merge=='left_only']

with so.Session(engine) as sess:
    for i in range(0, len(relationships_to_add), chunksize):
        to_add = relationships_to_add[i:i+chunksize].apply(make_relationship, axis=1)
        sess.add_all(list(to_add))
        sess.commit()

In [53]:
# due to misalignment with current athena version we have to accept gap in concept_id completeness even for classes that should have them

concept_stage_id[concept_stage_id.concept_id.isna() & ~concept_stage_id.concept_class_id.isin(non_omop_class)].head()

,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_code,valid_start_date,valid_end_date,invalid_reason,concept_id
393,Zinpentraxin alfa,drug,HemOnc,Component,,398,2019-05-27,2099-12-31,,NaN
616,MSI-H or dMMR Malignant solid neoplasm,condition,HemOnc,Condition,,624,2019-05-27,2099-12-31,,NaN
871,PMC4480917,,HemOnc,PMCID,,917,2019-05-27,2099-12-31,,NaN
878,PMC4504945,,HemOnc,PMCID,,928,2019-05-27,2099-12-31,,NaN
898,PMC1895015,,HemOnc,PMCID,,961,2019-05-27,2099-12-31,,NaN


In [54]:
# helper mapping data / merges within OMOP concepts
is_a = concept_relationship_stage_id[
    concept_relationship_stage_id.relationship_id=='Is a'
    ][['concept_code_1', 'concept_code_2', 'concept_id_1', 'concept_id_2']]


brand_mappings = concept_stage_id[
    concept_stage_id.concept_class_id=='Brand Name'
    ].merge(
        concept_relationship_stage_id[
            concept_relationship_stage_id.relationship_id=='Has brand name'
        ][['concept_code_1', 'concept_code_2', 'concept_id_1', 'concept_id_2']], 
        left_on='concept_code', 
        right_on='concept_code_2', 
        how='left'
    )

component_df = concept_stage_id[
    concept_stage_id.concept_class_id.isin(['Component', 'Procedure'])
    ].copy()

component_class_df = concept_stage_id[
    concept_stage_id.concept_class_id=='Component Class'
    ][['concept_name','concept_code','concept_id']].rename(
        columns={'concept_code': 'concept_code_2', 
                 'concept_name': 'concept_class_name'}
    )

has_modality = concept_relationship_stage_id[
    concept_relationship_stage_id.relationship_id=='Has modality'
    ][['concept_code_1', 'concept_code_2', 'concept_id_1', 'concept_id_1']]

modality_df = concept_stage_id[
    concept_stage_id.concept_class_id=='Modality'
    ][['concept_name','concept_code','concept_id']].rename(
        columns={'concept_code': 'concept_code_2', 
                 'concept_name': 'modality_name', 
                 'concept_id': 'concept_id_2'}
        )

In [55]:
route_mappings = {'44954': '26643006', 
                  '44957': '47625008',
                  '44979': '37161004',
                  '44994': '6064005',
                  '45080': '78421000',
                  '45153': '34206005',
                  '45215': '372471009',
                  '45273': '420254004',
                  '45426': '372466002',
                  '45531': '46713006',
                  '45574': '72607000',
                  '45684': '58100008',
                  '45939': '447694001'}

In [56]:
# helper data cleaning / transformation functions

def get_enum(e, s):
    try:
        return e[s.lower().strip().replace('-', '_').replace(' ', '_').replace('/', '_')]
    except:
        return None

def get_date(y, m, d):
    try:    
        datetime.date(int(y), int(m), int(d))
    except:
        return None

In [109]:
# functions to take dataframe rows and return database objects

def make_context(context_row):
    return Hemonc_Context(context_code = context_row.contextRaw,
                          context_name = context_row.contextPretty,
                          intent = get_enum(Intent, context_row.intent),
                          setting = get_enum(Setting, context_row.intent),
                          risk_stratification = get_enum(Risk, context_row.intent),
                          phenotype = get_enum(Phenotype, context_row.intent),
                          prior_therapy = get_enum(PriorTherapy, context_row.prior_therapy),
                          date_added = context_row.date_added)


def make_study(study_row):
    enrollment_from = get_date(study_row.from_year, study_row.from_month, study_row.from_day)
    enrol_to = get_date(study_row.to_year, study_row.to_month, study_row.to_day)
    return Hemonc_Study(study_code = study_row.study,
                        registry = study_row.registry,
                        trial_id = study_row.trial_id,
                        condition_code = study_row.concept_code,
                        enrollment_from = enrollment_from if not pd.isna(enrollment_from) else None,
                        enrollment_to = enrol_to if not pd.isna(enrol_to) else None,
                        phase = study_row.phase,
                        study_design = get_enum(StudyDesign, study_row.study_design),
                        study_design_imputed = study_row.study_design_imputed,
                        sact = study_row.sact if not pd.isna(study_row.sact) else None,
                        protocol = study_row.protocol,
                        fda_reg_study = False, #study_row.fda_reg_study,
                        fda_unreg_study = False, # study_row.fda_unreg_study,
                        start = study_row.start  if not pd.isna(study_row.start) else None,
                        end = study_row.end  if not pd.isna(study_row.end) else None,
                        study_group = study_row.study_group,
                        sponsor = study_row.sponsor,
                        date_added = study_row.date_added if not pd.isna(study_row.date_added) else None, 
                        date_modified = study_row.date_last_modified if not pd.isna(study_row.date_last_modified) else None
                        )

def make_regimen(regimen_row):
    return Hemonc_Regimen(regimen_cui = regimen_row.regimen_cui,
                          regimen_name = regimen_row.regimen)

def make_condition(condition_row):
    return Hemonc_Condition(condition_code = condition_row.concept_code,
                            condition_name = condition_row.concept_name,
                            condition_concept_id = int(condition_row.concept_id) if not pd.isna(condition_row.concept_id) else None)


def make_component(component_class_row):
    return Hemonc_Component(component_code = component_class_row.concept_code,
                            component_name = component_class_row.concept_name,
                            component_concept_id = int(component_class_row.concept_id) if not pd.isna(component_class_row.concept_id) else None)

def make_component_role(role_row):
    return Hemonc_Component_Role(regimen_cui = role_row.concept_code_1,
                                 component_code = role_row.concept_code_2 if role_row.component_class in ['Component', 'Procedure'] else None,
                                 component_class_code = role_row.concept_code_2 if role_row.component_class == 'Component Class' else None,
                                 relationship_id = role_row.relationship_id)


def make_component_class(component_class_row):
    return Hemonc_Component_Class(component_class_code = component_class_row.concept_code_2,
                                  component_class_name = component_class_row.concept_class_name,
                                  component_class_concept_id = int(component_class_row.concept_id) if not pd.isna(component_class_row.concept_id) else None)

def make_branch(branch_row):
    return Hemonc_Branch_Conditional(branch_name = branch_row.original,
                                     branch_type = branch_row.RULE_TYPE if not pd.isna(branch_row.RULE_TYPE) else BranchConditionalType.other,
                                     numeric_min = branch_row.MIN_NUM,
                                     numeric_max = branch_row.MAX_NUM,
                                     value = branch_row.RULE_VALUE)

def make_variant(variant_row):
    return Hemonc_Variant(variant_cui = variant_row.variant_cui,
                         variant_name = variant_row.variant,
                         regimen_cui = variant_row.regimen_cui)


def make_reg_part(reg_part_row):
    return Hemonc_Regimen_Part(variant_cui = reg_part_row.variant_cui,
                               regimen_part_id = reg_part_row.regimen_part_id,
                               portion = reg_part_row.portion if not pd.isna(reg_part_row.portion) else None,
                               cycle_sig_id = reg_part_row.cyclesigs,
                               timing = reg_part_row.timing,
                               timing_unit = reg_part_row.timing_unit
                               )

def assign_part_phase(reg_part_row):
    if not pd.isna(reg_part_row.phase):
        phases = [Part_Phase(regimen_part_id = reg_part_row.regimen_part_id,
                             variant_cui = reg_part_row.variant_cui, 
                             phase = get_enum(Phase, reg_part_row.phase),
                             ) for phase in reg_part_row.phase.split('|')]
        return phases

def make_cycle_sig(cs_row):
    return Hemonc_Cycle_Sig(cycle_sig_id = cs_row.cyclesigs,
                            duration_min = cs_row.DUR_MIN, 
                            duration_max = cs_row.DUR_MAX,
                            duration_units = cs_row.DUR_UNITS,
                            frequency_min = cs_row.FREQ_MIN,
                            frequency_max = cs_row.FREQ_MAX,
                            frequency_units = cs_row.FREQ_UNITS,
                            repeats_min = cs_row.REP_MIN,
                            repeats_max = cs_row.REP_MAX,
                            repeats_units = cs_row.REP_UNITS, 
                            cycle_len_min = cs_row.cycle_length_lb if not pd.isna(cs_row.cycle_length_lb) else None,
                            cycle_len_max = cs_row.cycle_length_ub if not pd.isna(cs_row.cycle_length_ub) else None,
                            cycle_len_units = cs_row.cycle_length_unit if not pd.isna(cs_row.cycle_length_unit) else None,
                            residual = ' '.join([t.text for t in cs_row.cs_residual]))


def make_modality(mod_row):
    return Hemonc_Modality(modality_code = mod_row.concept_code_2,
                           modality_name = mod_row.modality_name,
                           modality_concept_id = int(mod_row.concept_id_2) if not pd.isna(mod_row.concept_id_2) else None)

def make_ref(ref_row):
    return Hemonc_Ref(reference = ref_row.reference,
                      condition_code = ref_row.concept_code,
                      pmid = ref_row.pmid,
                      study = ref_row.study,
                      title = ref_row.title,
                      pmcid = ref_row.pmcid,
                      doi = ref_row.doi if not ref_row.doi=="" else None,
                      url = ref_row.url if not ref_row.url=="" else None,
                      journal = ref_row.journal,
                      biblio = ref_row.biblio,
                      pub_date = ref_row.pub_date,
                      order = ref_row.order,
                      update = ref_row['update'],
                      ref_type = ref_row.ref_type)

def make_sig(sig_row):
    return Hemonc_Sig(regimen_part_id = sig_row.regimen_part_id,
                      variant_cui = sig_row.variant_cui,
                      sig_id = sig_row.sig_id,
                      component_code = sig_row.concept_code,
                      component_name = sig_row.component,
                      component_role = get_enum(ComponentRole, sig_row.component_role),
                      step_number = sig_row.step_number,
                      component_class = sig_row['class'],
                      tail = sig_row['tail'],
                      route = sig_row.route,
                      doseMinNum = sig_row.doseMinNum if not pd.isna(sig_row.doseMinNum) else None,
                      doseMaxNum = sig_row.doseMaxNum if not pd.isna(sig_row.doseMaxNum) else None,
                      doseUnit = sig_row.doseUnit if not pd.isna(sig_row.doseUnit) else None,
                      doseCapNum = sig_row.doseCapNum if not pd.isna(sig_row.doseCapNum) else None,
                      doseCapUnit = sig_row.doseCapUnit if not pd.isna(sig_row.doseCapUnit) else None,
                      durationMinNum = sig_row.durationMinNum if not pd.isna(sig_row.durationMinNum) else None,
                      durationMaxNum = sig_row.durationMaxNum if not pd.isna(sig_row.durationMaxNum) else None,
                      durationUnit = sig_row.durationUnit if not pd.isna(sig_row.durationUnit) else None,
                      frequency = sig_row.frequency if not pd.isna(sig_row.frequency) else None,
                      inParens = sig_row.inParens if not pd.isna(sig_row.inParens) else None,
                      sequence = sig_row.sequence if not pd.isna(sig_row.sequence) else None,
                      seq_rel = sig_row['seq.rel'] if not pd.isna(sig_row['seq.rel']) else None,
                      seq_rel_what = sig_row['seq.rel.what'] if not pd.isna(sig_row['seq.rel.what']) else None)            


def make_sig_days(sd_row):
    days = '0' if pd.isna(sd_row.allDays) else sd_row.allDays
    return [Sig_Days(regimen_part_id = sd_row.regimen_part_id,
                     variant_cui = sd_row.variant_cui,
                     sig_id = sd_row.sig_id,
                     day = d) for d in days.split(',')]


# spacer matcher helper functions

small_num_lookup = {'one': 1, 'two': 2, 'three': 3, 'four': 4, 'five': 5, 'six': 6, 'seven': 7, 'eight': 8, 'nine': 9, 'ten': 10}

def safe_num(tok):
    try:
        return int(tok.text)
    except:
        try:
            return float(tok.text)
        except:
            try: 
                return small_num_lookup[tok.text.lower()]
            except:
                return None

def safe_age(numeric):
    ages = [safe_num(t) for t in numeric if t._.AGE]
    if len(ages) > 0:
        return ages[0]

def get_full_stopping_condition(doc, conditions):
    return doc[conditions[0].i:] if len(conditions) > 0 else None

def get_units(tokens):
    try:
        return [t.lemma_ for t in tokens if t._.TIMING_UNIT][0]
    except:
        return None

def get_min(group):
    try:
        return min([i for i in [safe_num(t) for t in group] if i])
    except:
        return None

def get_max(group): 
    try:
        if any([t._.GTE for t in group]):
            return -1
        return max([i for i in [safe_num(t) for t in group] if i])
    except:
        return None


In [85]:
role = ['Has cytotoxic chemo', 'Has targeted therapy', 'Has supportive med',
       'Has steroid tx', 'Has local therapy', 'Has immunosuppressor',
       'Has immunotherapy', 'Has endocrine tx', 'Has radiotherapy',
       'Has growth factor', 'Has AB-drug cjgt', 'Has radioconjugate',
       'Has antineoplastic', 'Has anticoag tx', 'Has pept-drug cjgt']

concept_reg_roles = (
    concept_relationship_stage_id[concept_relationship_stage_id.relationship_id.isin(role)][['concept_code_1', 'concept_id_1', 'relationship_id', 'concept_code_2', 'concept_id_2']]
        .merge(
                concept_stage_id[['concept_code', 'concept_name', 'concept_class_id']]
                .rename(
                    columns={'concept_code': 'concept_code_1', 
                             'concept_name': 'regimen_name',
                             'concept_class_id': 'regimen_class'}
                )
        )
        .merge(
                concept_stage_id[['concept_code', 'concept_name', 'concept_class_id']]
                .rename(
                    columns={'concept_code': 'concept_code_2', 
                             'concept_name': 'component_name',
                             'concept_class_id': 'component_class'}
                )
        )  
)

In [115]:
concept_reg_roles.head()

,concept_code_1,concept_id_1,relationship_id,concept_code_2,concept_id_2,regimen_name,regimen_class,component_name,component_class,role_object
0,795,35803428.0,Has immunosuppressor,122,35802975.0,Cyclophosphamide and Prednisolone,Regimen,Cyclophosphamide,Component,<hemonc_alchemy.model.hemonc_model.Hemonc_Comp...
1,795,35803428.0,Has immunosuppressor,417,35803267.0,Cyclophosphamide and Prednisolone,Regimen,Prednisolone,Component,<hemonc_alchemy.model.hemonc_model.Hemonc_Comp...
2,797,35803429.0,Has immunosuppressor,122,35802975.0,Cyclophosphamide and Prednisone,Regimen,Cyclophosphamide,Component,<hemonc_alchemy.model.hemonc_model.Hemonc_Comp...
3,797,35803429.0,Has immunosuppressor,418,35803268.0,Cyclophosphamide and Prednisone,Regimen,Prednisone,Component,<hemonc_alchemy.model.hemonc_model.Hemonc_Comp...
4,797,35803429.0,Has cytotoxic chemo,122,35802975.0,Cyclophosphamide and Prednisone,Regimen,Cyclophosphamide,Component,<hemonc_alchemy.model.hemonc_model.Hemonc_Comp...


In [86]:
concept_reg_roles['role_object'] = concept_reg_roles.apply(make_component_role, axis=1)

In [61]:
conditions = concept_stage_id[concept_stage_id.concept_class_id=='Condition'].apply(make_condition, axis=1)
component_class_df['component_class_object'] = component_class_df.apply(make_component_class, axis=1)

In [62]:
component_df['component_object'] = component_df.apply(make_component, axis=1)
components_with_class = component_df.merge(is_a, 
                                           left_on='concept_code', 
                                           right_on='concept_code_1').merge(component_class_df
                                                                            ).rename(columns={'concept_code_2': 'concept_class_code'})


In [63]:
modality_df['modality_object'] = modality_df.apply(make_modality, axis=1)

regimens_with_modality = concept_stage_id[
    concept_stage_id.invalid_reason == ''
    ].merge(
        has_modality, 
        left_on='concept_code', 
        right_on='concept_code_1'
    ).merge(
        modality_df
    ).rename(
            columns={'concept_code_2': 'concept_class_code'}
    ).drop_duplicates()


In [64]:
regimens_with_modality[regimens_with_modality[['concept_name', 'modality_name']].duplicated()]

,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_code,valid_start_date,valid_end_date,invalid_reason,concept_id,concept_code_1,concept_class_code,concept_id_1,concept_id_1,modality_name,concept_id_2,modality_object


In [65]:
components_with_class.apply(lambda x: x.component_object.component_classes.append(x.component_class_object), axis=1)

0    None
1    None
2    None
3    None
dtype: object

In [116]:
ref = ref.merge(concept_stage[concept_stage.concept_class_id=='Condition'][['concept_name', 'concept_code']], left_on='condition', right_on='concept_name', how='left')

In [117]:
ref['pub_date'] = pd.to_datetime(ref['pub.date'].map(lambda d: d.replace('D-', '').strip()))
ref['order'] = 1
ref['update'] = 1
ref_obj = ref[~ref.concept_code.isna()].apply(make_ref, axis=1)

In [118]:
# todo: confirm what is meant by a single cui with >1 title - is this a synonym?
# probably just different titles in different pages, as this is sourced from the pointer file originally.

regimen_dedup = regimen.drop_duplicates(subset='regimen_cui').copy()

In [119]:
context.date_added = pd.to_datetime(context.date_added.map(lambda d: d.replace('D-', '').strip()))
contexts = context.apply(make_context, axis=1)

In [120]:
# new issue: null condition codes - consider the fact that some of the 'conditions' being mapped to here are now actually regimen classes

# workaround mapping
study_condition_workaround = {
    'Stem cell mobilization regimens': 'Stem cell mobilization regimen', 
    'Allogeneic HSCT' : 'Allogeneic HSCT conditioning regimen',  
    'Cellular therapy conditioning regimens' : 'Cellular therapy conditioning regimen'
}

study.condition = study.condition.map(lambda x: study_condition_workaround[x] if x in study_condition_workaround else x)

In [121]:
# prepare study objects
study.date_added = pd.to_datetime(study.date_added, format='mixed', errors='coerce')
study.date_last_modified = pd.to_datetime(study.date_last_modified, format='mixed', errors='coerce')
enrol_dates = study.enrollment.str.split('to', expand=True)
enrol_from = enrol_dates[0].str.strip().str.split('-', expand=True).rename(columns={0: 'from_year', 1:'from_month', 2: 'from_day'}).fillna('01')
enrol_to = enrol_dates[1].str.strip().str.split('-', expand=True).rename(columns={0: 'to_year', 1:'to_month', 2: 'to_day'}).fillna('01')
study = pd.concat([study, enrol_from, enrol_to], axis=1)
study = study.merge(concept_stage[concept_stage.concept_class_id.isin(['Condition', 'Regimen Class'])][['concept_name', 'concept_code']], left_on='condition', right_on='concept_name', how='left')
study.start = pd.to_datetime(study.start, format='mixed', errors='coerce')
study.end = pd.to_datetime(study.end, format='mixed', errors='coerce')

In [130]:
study['date_added'] = '2099-12-31'

In [131]:
study.date_added = pd.to_datetime(study.date_added)

In [132]:
study['study_object'] = study.apply(make_study, axis=1)
regimen_dedup['regimen_object'] = regimen_dedup.apply(make_regimen, axis=1)

In [96]:
def append_modality(regimen_obj, modality_obj):
    if not pd.isna(modality_obj):
        regimen_obj.modalities.append(modality_obj)

In [123]:
regimen_dedup[regimen_dedup.regimen_cui.duplicated(keep=False)]

,regimen,regimen_cui,regimen_object


In [135]:
study.columns

Index(['study', 'registry', 'trial_id', 'condition', 'condition_cui',
       'enrollment', 'phase', 'study_design', 'study_design_imputed', 'sact',
       'protocol', 'reg_study', 'unreg_study', 'start', 'end', 'study_group',
       'sponsor', 'sponsor_type', 'temp', 'date_added', 'date_last_modified',
       'from_year', 'from_month', 'from_day', 'to_year', 'to_month', 'to_day',
       'concept_name', 'concept_code', 'study_object'],
      dtype='object')

In [ ]:
study[]

,study,registry,trial_id,condition,condition_cui,enrollment,phase,study_design,study_design_imputed,sact,...,date_last_modified,from_year,from_month,from_day,to_year,to_month,to_day,concept_name,concept_code,study_object
0,KN035-BTC,ClinicalTrials.gov,NCT03478488,Cholangiocarcinoma,580,,,CBD,False,NaN,...,NaT,,01,01,01,01,01,Cholangiocarcinoma,580,<hemonc_alchemy.model.hemonc_model.Hemonc_Stud...
1,KN035-CN-006,ClinicalTrials.gov,NCT03667170,MSI-H or dMMR Malignant solid neoplasm,624,2018-2019,Phase 2,Non-randomized,False,True,...,NaT,2018,2019,01,01,01,01,MSI-H or dMMR Malignant solid neoplasm,624,<hemonc_alchemy.model.hemonc_model.Hemonc_Stud...
2,AMPECT,ClinicalTrials.gov,NCT02494570,PEComa,37829,2016-2018,Phase 2,Non-randomized,False,True,...,NaT,2016,2018,01,01,01,01,PEComa,37829,<hemonc_alchemy.model.hemonc_model.Hemonc_Stud...
3,AB06006,ClinicalTrials.gov,NCT00814073,Systemic mastocytosis,669,2009-2015,Phase 3,CBD,False,True,...,NaT,2009,2015,01,01,01,01,Systemic mastocytosis,669,<hemonc_alchemy.model.hemonc_model.Hemonc_Stud...
4,AB07001,ClinicalTrials.gov,NCT01506336,Gastrointestinal stromal tumor,602,2009-2011,Randomized phase 2,CBD,False,True,...,NaT,2009,2011,01,01,01,01,Gastrointestinal stromal tumor,602,<hemonc_alchemy.model.hemonc_model.Hemonc_Stud...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7394,FOCUS_gastric,ClinicalTrials.gov,NCT01364376,Gastric cancer,601,2011-2016,Phase 3,In-class switch,False,True,...,NaT,2011,2016,01,01,01,01,Gastric cancer,601,<hemonc_alchemy.model.hemonc_model.Hemonc_Stud...
7395,ZJCH-E-E,ClinicalTrials.gov,NCT02813967,Esophageal cancer,595,2016-06-01 to 2018-08-31,Phase 3,Escalation,False,True,...,NaT,2016,06,01,2018,08,31,Esophageal cancer,595,<hemonc_alchemy.model.hemonc_model.Hemonc_Stud...
7396,CD7-001,ClinicalTrials.gov,NCT04599556,Cellular therapy conditioning regimen,114735,2021-11 to 2023-09,Phase 1,Non-randomized,False,True,...,NaT,2021,11,01,2023,09,01,Cellular therapy conditioning regimen,114735,<hemonc_alchemy.model.hemonc_model.Hemonc_Stud...
7397,CTTNKTL-III/IV,ClinicalTrials.gov,NCT01501149,NK- and T-cell lymphoma,637,2011-01 to 2019-02,Phase 4,In-class switch,False,True,...,NaT,2011,01,01,2019,02,01,NK- and T-cell lymphoma,637,<hemonc_alchemy.model.hemonc_model.Hemonc_Stud...


In [139]:
with so.Session(engine) as sess:
    sess.add_all(conditions)
    sess.add_all(list(component_class_df.component_class_object))
    sess.add_all(list(component_df.component_object))
    sess.add_all(contexts)
    sess.add_all(list(study[~study.concept_code.isna()&~study.duplicated(subset=['condition', 'trial_id', 'study'])].study_object))
    sess.add_all(list(modality_df.modality_object))
    sess.add_all(list(regimen_dedup.regimen_object))
    sess.add_all(list(ref_obj))
    sess.add_all(list(concept_reg_roles.role_object))
    sess.commit()

In [140]:
with so.Session(engine) as sess:
    regimen_objects = pd.DataFrame(sess.query(Hemonc_Regimen), columns=['regimen_object'])
    regimen_objects['regimen_cui'] = regimen_objects.regimen_object.map(lambda x: str(x.regimen_cui))
    modality_objects = pd.DataFrame(sess.query(Hemonc_Modality), columns=['modality_object'])
    modality_objects['modality_code'] = modality_objects.modality_object.map(lambda x: str(x.modality_code))
    modality_lookup = regimens_with_modality[['concept_code', 'concept_class_code']].drop_duplicates()
    modality_lookup = (
        modality_lookup
        .merge(regimen_objects, left_on='concept_code', right_on='regimen_cui')
        .merge(modality_objects, left_on='concept_class_code', right_on='modality_code')
    )
    modality_lookup.apply(lambda row: row.regimen_object.modalities.append(row.modality_object), axis=1)
    sess.commit()

In [163]:
# we are not going to pull in out of date variant versions
sig_vars = sigs.merge(variant[variant.version=='1'][['variant_cui']])

In [164]:
for (r, c), dets in sig_vars[~sig_vars.step_number.str.contains('1 of') & sig_vars.allDays.isna()].groupby(['regimen', 'component']):
    if len(dets)>1:
        break

In [165]:
sig_vars

,study,regimen,regimen_cui,phase,portion,component,component_cui,component_role,cycle_length_lb,cycle_length_ub,...,sequence,seq.rel.when,seq.rel.when.unit,seq.rel,seq.rel.what,tail,variant,variant_cui,temp,date_added
0,MDACC ID01-233,"(90)YFC, then allo HSCT",19726,Consolidation,-,Allogeneic stem cells,38881,primary systemic,1,1,...,NaN,NaN,NaN,NaN,NaN,-,Variant #01,129496,NaN,D-2023-09-19
1,MDACC ID01-233,"(90)YFC, then allo HSCT",19726,Consolidation,-,Cyclophosphamide,122,primary systemic,1,1,...,NaN,NaN,NaN,NaN,NaN,-,Variant #01,129496,NaN,D-2023-09-19
2,MDACC ID01-233,"(90)YFC, then allo HSCT",19726,Consolidation,-,Fludarabine,224,primary systemic,1,1,...,NaN,NaN,NaN,NaN,NaN,-,Variant #01,129496,NaN,D-2023-09-19
3,MDACC ID01-233,"(90)YFC, then allo HSCT",19726,Consolidation,-,Ibritumomab tiuxetan,262,primary systemic,1,1,...,NaN,NaN,NaN,NaN,NaN,-,Variant #01,129496,NaN,D-2023-09-19
4,MDACC ID01-233,"(90)YFC, then allo HSCT",19726,Consolidation,-,Ibritumomab tiuxetan,262,primary systemic,1,1,...,NaN,NaN,NaN,NaN,NaN,-,Variant #01,129496,NaN,D-2023-09-19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18755,eNRGy,Zenocutuzumab monotherapy,150750,NaN,-,Zenocutuzumab,150749,primary systemic,14,14,...,NaN,NaN,NaN,NaN,NaN,-,Variant #01,151032,NaN,D-2025-03-25
18756,YH-S001-04,Zimberelimab monotherapy,114298,NaN,-,Zimberelimab,48632,primary systemic,14,14,...,NaN,NaN,NaN,NaN,NaN,-,Variant #01,134403,NaN,D-2023-09-19
18757,PRO-2580,ZRd,27764,NaN,-,Dexamethasone,156,primary systemic,28,28,...,NaN,NaN,NaN,NaN,NaN,-,Variant #01,134404,NaN,D-2023-09-19
18758,PRO-2580,ZRd,27764,NaN,-,Lenalidomide,299,primary systemic,28,28,...,NaN,NaN,NaN,NaN,NaN,-,Variant #01,134404,NaN,D-2023-09-19


In [143]:
# parsing out branch conditions with spacy matchers

branch_crit = sig_vars.branch.value_counts().reset_index()

all_branches = []

for crit in branch_crit.branch.unique():
    combined_branches = crit.split('AND ')
    for br in combined_branches:
        all_branches += [b.lower().strip() for b in br.split('OR ')]

all_branches = list(set(all_branches))

branch_details = pd.DataFrame({'original': all_branches, 'doc': [nlp(b) for b in all_branches]})

for label, config in rules.items():
    branch_details.doc.apply(lambda d: match_entities(matchers[label], d, label, config["MERGE"]))
    branch_details[label] = branch_details.doc.apply(lambda d: [t for t in d if getattr(t._, label)])
    branch_details[f'has_{label}'] = (branch_details[label].apply(len)>0)

branch_details['MODIFIER_HEAD'] = branch_details.doc.apply(get_modifier_child)
branch_details['NOUNS'] = branch_details.doc.apply(get_nouns)
branch_details['NUMERIC'] = branch_details.doc.map(lambda doc: [tok for tok in doc if tok.like_num])

In [144]:
# parse numeric branch factors into constituant elements

branch_details.loc[branch_details.has_AGE, 'RULE_TYPE'] = BranchConditionalType.age
branch_details.loc[branch_details.has_SIZE, 'RULE_TYPE'] = BranchConditionalType.size
branch_details.loc[branch_details.has_LAB, 'RULE_TYPE'] = BranchConditionalType.lab
branch_details.loc[branch_details.has_STAGE, 'RULE_TYPE'] = BranchConditionalType.stage

branch_details.loc[branch_details.has_STAGE, 'RULE_VALUE'] = branch_details.STAGE.apply(lambda x: ' '.join([tok.text for tok in x]) if len(x) > 0 else None)

num = branch_details.NUMERIC.apply(lambda x: safe_num(x[0]) if len(x) > 0 else None)
age = branch_details.NUMERIC.apply(lambda x: safe_age(x))
first_num = branch_details.NUMERIC.apply(lambda x: safe_num(x[0]) if len(x) > 0 else None)
second_num = branch_details.NUMERIC.apply(lambda x: safe_num(x[1]) if len(x) > 1 else None)

gt = (branch_details.GT.apply(len)>0)
gte = (branch_details.GTE.apply(len)>0)
lt = (branch_details.LT.apply(len)>0)
lte = (branch_details.LTE.apply(len)>0)

r = (branch_details.RANGE.apply(len)>0)

branch_details.loc[branch_details.has_AGE & gt, 'MIN_NUM'] = age.apply(lambda x: x + 1 if x else None)
branch_details.loc[branch_details.has_AGE & gte, 'MIN_NUM'] = age
branch_details.loc[branch_details.has_AGE & lt, 'MAX_NUM'] = age.apply(lambda x: x - 1 if x else None)
branch_details.loc[branch_details.has_AGE & lte, 'MAX_NUM'] = age

branch_details.loc[(branch_details.has_SIZE | branch_details.has_LAB) & gt, 'MIN_NUM'] = num.apply(lambda x: x + 1 if x else None)
branch_details.loc[(branch_details.has_SIZE | branch_details.has_LAB) & gte, 'MIN_NUM'] = num
branch_details.loc[(branch_details.has_SIZE | branch_details.has_LAB) & lt, 'MAX_NUM'] = num.apply(lambda x: x - 1 if x else None)
branch_details.loc[(branch_details.has_SIZE | branch_details.has_LAB) & lte, 'MAX_NUM'] = num

branch_details.loc[(branch_details.has_AGE | branch_details.has_SIZE | branch_details.has_LAB) & r, 'MIN_NUM'] = first_num
branch_details.loc[(branch_details.has_AGE | branch_details.has_SIZE | branch_details.has_LAB) & r, 'MAX_NUM'] = second_num

ValueError: cannot set a frame with no defined index and a scalar

In [ ]:
# very likely to be able to improve these value extraction steps through either medspacy NER or better rules / more fullsome parsing, but a decent first pass

branch_details.loc[branch_details.RULE_VALUE.isna(), 'RULE_VALUE'] = branch_details.apply(lambda row: ' '.join([t.text for t in (row.FACT_MODIFIER + row.FACT)]), axis=1)

In [ ]:
branches = branch_details.apply(make_branch, axis=1)

In [ ]:
with so.Session(engine) as sess:
    sess.add_all(branches)
    sess.commit()

In [148]:
variant.regimen_cui = variant.regimen_cui.astype(int)

In [149]:
variant_study = variant.merge(pointer[['regimen', 'regimen_cui']].drop_duplicates(), how='left')[['regimen', 'study', 'variant_cui', 'variant', 'regimen_cui']].drop_duplicates()
variant_study['variant_object'] = variant_study.apply(make_variant, axis=1)

In [150]:
study_object_lookup = {s.study: s.study_object for s in study.itertuples()}

In [151]:
# new issue - some null studies in variant study map
variant[variant.study.isna()]

for row in variant_study[~variant_study.study.isna()].itertuples():
    for study_name in row.study.split('|'):
        try:
            row.variant_object.studied_in.append(study_object_lookup[study_name])
        except:
            print(f'missing study: {study_name} ({row.regimen})')

missing study: No supporting study (Afatinib monotherapy)
missing study: No supporting study (Atezolizumab monotherapy)
missing study: No supporting study (Atezolizumab monotherapy)
missing study: No supporting study (Atezolizumab monotherapy)
missing study: No supporting study (Atezolizumab monotherapy)
missing study: No supporting study (Axitinib and Avelumab)
missing study:  (Bivalirudin monotherapy)
missing study:  (Brachytherapy protocol)
missing study: No supporting study (Cabozantinib and Nivolumab)
missing study: No supporting study (Cabozantinib monotherapy)
missing study:  (Capecitabine, Paclitaxel, RT)
missing study:  (Capecitabine, Paclitaxel, RT)
missing study:  (Capecitabine, Paclitaxel, RT)
missing study:  (Capecitabine, Paclitaxel, RT)
missing study:  (COG AHEP0731 protocol F)
missing study:  (COG AHEP0731 protocol F)
missing study:  (COG AHEP0731 protocol H - non-responders)
missing study:  (COG AHEP0731 protocol H - non-responders)
missing study:  (COG AHEP0731 protoc

In [167]:
regimen_part_sig = sig_vars[['regimen', 'variant', 'phase', 'portion', 'cyclesigs', 'timing', 'branch', 'cycle_length_lb', 'cycle_length_ub', 'cycle_length_unit']].drop_duplicates()

In [168]:
sig_vars

,study,regimen,regimen_cui,phase,portion,component,component_cui,component_role,cycle_length_lb,cycle_length_ub,...,sequence,seq.rel.when,seq.rel.when.unit,seq.rel,seq.rel.what,tail,variant,variant_cui,temp,date_added
0,MDACC ID01-233,"(90)YFC, then allo HSCT",19726,Consolidation,-,Allogeneic stem cells,38881,primary systemic,1,1,...,NaN,NaN,NaN,NaN,NaN,-,Variant #01,129496,NaN,D-2023-09-19
1,MDACC ID01-233,"(90)YFC, then allo HSCT",19726,Consolidation,-,Cyclophosphamide,122,primary systemic,1,1,...,NaN,NaN,NaN,NaN,NaN,-,Variant #01,129496,NaN,D-2023-09-19
2,MDACC ID01-233,"(90)YFC, then allo HSCT",19726,Consolidation,-,Fludarabine,224,primary systemic,1,1,...,NaN,NaN,NaN,NaN,NaN,-,Variant #01,129496,NaN,D-2023-09-19
3,MDACC ID01-233,"(90)YFC, then allo HSCT",19726,Consolidation,-,Ibritumomab tiuxetan,262,primary systemic,1,1,...,NaN,NaN,NaN,NaN,NaN,-,Variant #01,129496,NaN,D-2023-09-19
4,MDACC ID01-233,"(90)YFC, then allo HSCT",19726,Consolidation,-,Ibritumomab tiuxetan,262,primary systemic,1,1,...,NaN,NaN,NaN,NaN,NaN,-,Variant #01,129496,NaN,D-2023-09-19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18755,eNRGy,Zenocutuzumab monotherapy,150750,NaN,-,Zenocutuzumab,150749,primary systemic,14,14,...,NaN,NaN,NaN,NaN,NaN,-,Variant #01,151032,NaN,D-2025-03-25
18756,YH-S001-04,Zimberelimab monotherapy,114298,NaN,-,Zimberelimab,48632,primary systemic,14,14,...,NaN,NaN,NaN,NaN,NaN,-,Variant #01,134403,NaN,D-2023-09-19
18757,PRO-2580,ZRd,27764,NaN,-,Dexamethasone,156,primary systemic,28,28,...,NaN,NaN,NaN,NaN,NaN,-,Variant #01,134404,NaN,D-2023-09-19
18758,PRO-2580,ZRd,27764,NaN,-,Lenalidomide,299,primary systemic,28,28,...,NaN,NaN,NaN,NaN,NaN,-,Variant #01,134404,NaN,D-2023-09-19


In [169]:
# spacy matcher parsing of cycle sigs

cycle_sigs_parsed = regimen_part_sig[['cyclesigs', 'cycle_length_lb', 'cycle_length_ub', 'cycle_length_unit']].fillna('').drop_duplicates().copy()
cycle_sigs_parsed['doc'] = cycle_sigs_parsed.cyclesigs.map(nlp)

for label, config in rules.items():
    cycle_sigs_parsed.doc.apply(lambda d: match_entities(matchers[label], d, label, config["MERGE"]))
    cycle_sigs_parsed[label] = cycle_sigs_parsed.doc.apply(lambda d: [t for t in d if getattr(t._, label)])
    cycle_sigs_parsed[f'has_{label}'] = (cycle_sigs_parsed[label].apply(len)>0)

cycle_sigs_parsed['full_stopping'] = cycle_sigs_parsed.apply(lambda row: get_full_stopping_condition(row.doc, row.STOPPING_CONDITION), axis=1)


cycle_sigs_parsed['DUR_MIN'] = cycle_sigs_parsed.DURATION.apply(get_min)
cycle_sigs_parsed['DUR_MAX'] = cycle_sigs_parsed.DURATION.apply(get_max)
cycle_sigs_parsed['DUR_UNITS'] = cycle_sigs_parsed.DURATION.apply(get_units)
cycle_sigs_parsed['FREQ_MIN'] = cycle_sigs_parsed.FREQUENCY.apply(get_min)
cycle_sigs_parsed['FREQ_MAX'] = cycle_sigs_parsed.FREQUENCY.apply(get_max)
cycle_sigs_parsed['FREQ_UNITS'] = cycle_sigs_parsed.FREQUENCY.apply(get_units)
cycle_sigs_parsed['REP_MIN'] = cycle_sigs_parsed.REPEATS.apply(get_min)
cycle_sigs_parsed['REP_MAX'] = cycle_sigs_parsed.REPEATS.apply(get_max)
cycle_sigs_parsed['REP_UNITS'] = cycle_sigs_parsed.REPEATS.apply(get_units)

In [170]:
reg_parts = sig_vars[['regimen', 'variant_cui','variant','phase', 'portion', 'cyclesigs', 'timing', 'timing_unit']].drop_duplicates().copy()
reg_parts['regimen_part_id'] = reg_parts.sort_values(['variant_cui', 'phase', 'portion', 'cyclesigs', 'timing', 'timing_unit']).groupby('variant_cui').cumcount()

In [171]:
reg_parts[reg_parts.cyclesigs.str.contains('2 and', na=False, case=False)]

,regimen,variant_cui,variant,phase,portion,cyclesigs,timing,timing_unit,regimen_part_id
7433,Dasatinib and Blinatumomab,131515,Variant #01,Consolidation,-,42-day cycle for at least 2 and 5 cycles,NaN,Cycle,0


In [172]:
regimen_parts = reg_parts.apply(make_reg_part, axis=1)
part_phase = reg_parts.apply(assign_part_phase, axis=1)
cycle_sigs_parsed['cs_residual'] = cycle_sigs_parsed.doc.apply(lambda d: [t for t in d if not t._.FREQUENCY and not t._.REPEATS and not t._.DURATION and not t._.GTE and t.text != 'for'])

In [173]:
cycle_sigs_parsed[cycle_sigs_parsed.cs_residual.apply(len) > 0]

,cyclesigs,cycle_length_lb,cycle_length_ub,cycle_length_unit,doc,AGE,has_AGE,STAGE,has_STAGE,RANGE,...,DUR_MIN,DUR_MAX,DUR_UNITS,FREQ_MIN,FREQ_MAX,FREQ_UNITS,REP_MIN,REP_MAX,REP_UNITS,cs_residual
186,Monthly cycles,1,1,month,"(Monthly, cycles)",[],False,[],False,[],...,NaN,NaN,None,NaN,NaN,None,NaN,NaN,None,"[Monthly, cycles]"
619,Continued indefinitely,1,NUB,indeterminate,"(Continued, indefinitely)",[],False,[],False,[],...,NaN,NaN,None,NaN,NaN,None,NaN,NaN,None,"[Continued, indefinitely]"
623,Continued indefinitely unless radiotherapy pla...,,,,"(Continued, indefinitely, unless, radiotherapy...",[],False,[],False,[],...,NaN,NaN,None,NaN,NaN,None,NaN,NaN,None,"[Continued, indefinitely, unless, radiotherapy..."
702,Continued until achievement of CR or best resp...,1,1,week,"(Continued, until, achievement, of, CR, or, be...",[],False,[],False,[],...,NaN,NaN,None,NaN,NaN,None,NaN,NaN,None,"[Continued, until, achievement, of, CR, or, be..."
851,Continued until CR or 60 days,c,60,day,"(Continued, until, CR, or, 60, days)",[],False,[],False,[],...,NaN,NaN,None,NaN,NaN,None,NaN,NaN,None,"[Continued, until, CR, or, 60, days]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18256,Duration of treatment not specified,1,1,indeterminate,"(Duration, of, treatment, not, specified)",[],False,[],False,[],...,NaN,NaN,None,NaN,NaN,None,NaN,NaN,None,"[Duration, of, treatment, not, specified]"
18292,14-day lead-in,14,14,day,"(14, -, day, lead, -, in)",[],False,[],False,[],...,NaN,NaN,None,NaN,NaN,None,NaN,NaN,None,"[14, -, day, lead, -, in]"
18614,21-day cycles until progression or two cycles ...,21,21,day,"(21, -, day, cycles, until, progression, or, t...",[],False,[],False,[],...,NaN,NaN,None,21.0,21.0,day,2.0,2.0,cycle,"[until, progression, or, past, documented, CR]"
18646,14-day cycle for 2 years or until disease prog...,14,14,day,"(14, -, day, cycle, for, 2, years, or, until, ...",[],False,[],False,[],...,NaN,NaN,None,14.0,14.0,day,NaN,NaN,None,"[2, years, or, until, disease, progression, or..."


In [174]:
cs = cycle_sigs_parsed.apply(make_cycle_sig, axis=1)

In [175]:
# new issue - variant_cui not unique in this file any more - need to resolve the study field to unique pairs?
variant_study[variant_study.variant_cui.duplicated(keep=False)]

,regimen,study,variant_cui,variant,regimen_cui,variant_object
10,131Iodine-Tositumomab monotherapy,CP-97-012|Kaminski et al. 1993|Kaminski et al....,144834,Variant #02,19847,<hemonc_alchemy.model.hemonc_model.Hemonc_Vari...
55,7+3d,Arlin et al. 1990|AZA-AML-001|CALGB 7421|CALGB...,129508,Variant #05,814,<hemonc_alchemy.model.hemonc_model.Hemonc_Vari...
58,7+3d,Jin et al. 2013|Arlin et al. 1990|AZA-AML-001|...,129508,Variant #05,814,<hemonc_alchemy.model.hemonc_model.Hemonc_Vari...
70,7+3d,AML-AZA|AML2003|AML2006|AZA-AML-001|BRIGHT AML...,129512,Variant #09,814,<hemonc_alchemy.model.hemonc_model.Hemonc_Vari...
73,7+3d,AML-AZA|AML2003|AML2006|AZA-AML-001|CALGB 1020...,129512,Variant #09,814,<hemonc_alchemy.model.hemonc_model.Hemonc_Vari...
...,...,...,...,...,...,...
17768,Vemurafenib monotherapy,BRIM-3|BRIM2|coBRIM|COLUMBUS|COMBI-v|Flaherty ...,134277,Variant #04,5142,<hemonc_alchemy.model.hemonc_model.Hemonc_Vari...
17906,Vinorelbine monotherapy,E7389-C086-304|Furuse et al. 1996|Gasparini et...,134312,Variant #06,6944,<hemonc_alchemy.model.hemonc_model.Hemonc_Vari...
17916,Vinorelbine monotherapy,E7389-C086-304|Furuse et al. 1996|Furuse et al...,134312,Variant #06,6944,<hemonc_alchemy.model.hemonc_model.Hemonc_Vari...
18151,Warfarin monotherapy,AMPLIFY|CATCH|EINSTEIN Acute DVT|EINSTEIN-PE|H...,134394,Variant #02,3531,<hemonc_alchemy.model.hemonc_model.Hemonc_Vari...


In [176]:
# new issue - handling null condition codes
variant_study[variant_study.study.notna() & ~variant_study.study.isin(study[study.concept_code.isna()].study.unique())].drop_duplicates(subset='variant_cui')

,regimen,study,variant_cui,variant,regimen_cui,variant_object
0,"(90)YFC, then allo HSCT",MDACC ID01-233,129496,Variant #01,19726,<hemonc_alchemy.model.hemonc_model.Hemonc_Vari...
5,131Iodine-Tositumomab monotherapy,CP-97-012|Kaminski et al. 1993|Kaminski et al....,129497,Variant #01,19847,<hemonc_alchemy.model.hemonc_model.Hemonc_Vari...
10,131Iodine-Tositumomab monotherapy,CP-97-012|Kaminski et al. 1993|Kaminski et al....,144834,Variant #02,19847,<hemonc_alchemy.model.hemonc_model.Hemonc_Vari...
12,4d+7,ALFA 9803,150895,Variant #01,110391,<hemonc_alchemy.model.hemonc_model.Hemonc_Vari...
13,4i+7,ALFA 9803,150896,Variant #01,110392,<hemonc_alchemy.model.hemonc_model.Hemonc_Vari...
...,...,...,...,...,...,...
18180,Zanubrutinib monotherapy,BGB-3111-206|BGB-3111-AU-003|BRUIN MCL-321,134401,Variant #02,50483,<hemonc_alchemy.model.hemonc_model.Hemonc_Vari...
18181,Zanubrutinib monotherapy,No supporting study,134402,Variant #03,50483,<hemonc_alchemy.model.hemonc_model.Hemonc_Vari...
18186,Zenocutuzumab monotherapy,eNRGy,151032,Variant #01,150750,<hemonc_alchemy.model.hemonc_model.Hemonc_Vari...
18187,Zimberelimab monotherapy,YH-S001-04,134403,Variant #01,114298,<hemonc_alchemy.model.hemonc_model.Hemonc_Vari...


In [177]:
from collections import defaultdict
vs_lookup = defaultdict(list)

for variant_map in variant_study[['variant_cui', 'study']].fillna('').itertuples():
    vs_lookup[variant_map.variant_cui] += variant_map.study.split('|')

In [178]:
# new issue - variant_cui - study id is not unique in this file any more - need to resolve the study field to unique pairs?

dupe_vs_map = [k for k, v in vs_lookup.items() if len(v) != len(set(v))]

In [179]:
variant_study[variant_study.variant_cui == 129495]#.variant_object.iloc[0].__dict__

,regimen,study,variant_cui,variant,regimen_cui,variant_object


In [180]:
with so.Session(engine) as sess:
    sess.add_all(list(variant_study[variant_study.study.notna() & 
                                    ~variant_study.study.isin(study[study.concept_code.isna()].study.unique()) & 
                                    ~variant_study.variant_cui.isin(dupe_vs_map)].drop_duplicates(subset='variant_cui').variant_object))
    sess.add_all(regimen_parts)
    sess.add_all(cs)
    sess.commit()

/var/folders/jy/szty1pk12tl__kzv4ybc_95r0000gp/T/ipykernel_26789/182496178.py:7: SAWarning: Object of type <Hemonc_Variant> not in session, add operation along 'Hemonc_Study.studies' won't proceed
  sess.commit()


In [181]:
sig_vars[sig_vars.variant_cui==131430]#sig_vars[['regimen', 'variant', 'phase', 'portion', 'cyclesigs', 'timing', 'branch', 'cycle_length_lb', 'cycle_length_ub', 'cycle_length_unit']].duplicated()]

regimen_part_sig[regimen_part_sig.regimen=='D-FEC plus Bev']

reg_parts[reg_parts.variant_cui==131430]#sig_vars[['regimen', 'variant', 'phase', 'portion', 'cyclesigs', 'timing', 'branch', 'cycle_length_lb', 'cycle_length_ub', 'cycle_length_unit']].duplicated()]


,regimen,variant_cui,variant,phase,portion,cyclesigs,timing,timing_unit,regimen_part_id


In [182]:
sig_vars_components = sig_vars.merge(components_with_class[['concept_name', 'concept_code']], left_on='component', right_on='concept_name', how='left').drop_duplicates().merge(reg_parts[['variant_cui', 'phase', 'portion', 'cyclesigs', 'regimen_part_id']])
sig_vars_components[['variant_cui', 'regimen_part_id', 'component', 'component_role', 'timing', 'step_number', 'inParens', 'class', 'tail']][sig_vars_components[['variant_cui', 'regimen_part_id', 'component', 'component_role', 'timing', 'step_number', 'class', 'tail']].duplicated()]

,variant_cui,regimen_part_id,component,component_role,timing,step_number,inParens,class,tail
7226,136544,0,Cyclosporine,primary systemic,NaN,1 of 1,NaN,Non-canonical Sig,adjusted to maintain therapeutic cyclosporine ...
7330,131390,0,Cytarabine,primary systemic,NaN,1 of 1,NaN,IV intermittent canonical Sig,-
12964,136926,0,Hydroxyurea,primary systemic,NaN,1 of 1,NaN,Non-canonical Sig,-
12965,136926,0,Hydroxyurea,primary systemic,NaN,1 of 1,NaN,Non-canonical Sig,-
12966,136926,0,Hydroxyurea,primary systemic,NaN,1 of 1,NaN,Non-canonical Sig,-
12967,136926,0,Hydroxyurea,primary systemic,NaN,1 of 1,NaN,Non-canonical Sig,-
12968,136926,0,Hydroxyurea,primary systemic,NaN,1 of 1,NaN,Non-canonical Sig,-
14663,132911,0,Lenalidomide,primary systemic,NaN,1 of 1,NaN,Non-IV canonical Sig,-
14664,132911,0,Lenalidomide,primary systemic,NaN,1 of 1,NaN,Non-IV canonical Sig,-
15438,144895,0,Methotrexate,primary systemic,NaN,1 of 1,NaN,Non-IV canonical Sig,-


In [183]:
sig_vars_components['sig_id'] = sig_vars_components.groupby(['regimen', 'variant_cui', 'regimen_part_id']).cumcount()

In [184]:
sig_vars_components = sig_vars_components.sort_values(['variant_cui', 'regimen_part_id', 'concept_name', 'step_number'])

In [185]:
# if days are empty, but this component and regimen part are identical to the previous row when sorted in this fashion, filling these day details forward

sig_vars_components.loc[sig_vars_components.allDays.isna() & (sig_vars_components.concept_name.shift(1) ==  sig_vars_components.concept_name), 'allDays'] = sig_vars_components.allDays.shift(1)

In [186]:
sig_objects = sig_vars_components.apply(make_sig, axis=1)
sd = sig_vars_components.apply(make_sig_days, axis=1)

In [187]:
sig_vars_components.component.apply(len).max()

47

In [188]:
with so.Session(engine) as sess:
    sess.add_all(sig_objects)
    for s in sd:
        sess.add_all(s)
    sess.commit()

In [189]:
sig_vars_components[['component', 'doseMinNum', 'doseMaxNum', 'doseUnit', 'doseCapNum', 'doseCapUnit', 'divided', 'durationMinNum', 'durationMaxNum', 'durationUnit', 'frequency', 'inParens', 'sequence', 'seq.rel', 'seq.rel.what']]['seq.rel.what'].value_counts()

seq.rel.what
Elotuzumab           12
Ifosfamide            6
Leucovorin            5
Fludarabine           4
Rituximab             3
Irinotecan            3
Radiation therapy     2
HIPEC                 2
Melphalan             2
Methotrexate          1
Goserelin             1
Teniposide            1
chemotherapy          1
Ultraviolet A         1
Etoposide             1
Mesna                 1
Radiotherapy          1
Levoleucovorin        1
Cyclophosphamide      1
Pemetrexed            1
Name: count, dtype: int64

In [190]:
regimen[regimen.regimen_cui.duplicated()].head()

,regimen,regimen_cui
190,"Busulfan and Fludarabine, then allo HSCT",1584
264,CYVE,1105
624,"Arsenic trioxide, then ATRA and Daunorubicin",2391
817,FC,2913
852,"Fludarabine, Busulfan, ATG, Ibritumomab tiuxetan",3015


In [191]:
### Issues

# Need a n:m context <-> status mapper

# single record with >1 setting - is this a variant?
context[context.setting.map(lambda x: get_enum(Setting, x)).isna()].setting.value_counts()

# list of regimens that have no associated variants

vv = variant.merge(pointer[['regimen', 'regimen_cui']].drop_duplicates(), how='outer')
vv[vv.variant.isna()].regimen.unique()

# and vice-versa
vv[vv.regimen_cui.isna()]


# some potential duplication across variant/sig rows
dup = sig_vars[sig_vars[['study',  'regimen', 'component', 'variant_cui', 'branch', 'timing', 'step_number', 'portion', 'class']].duplicated(keep=False)]

for regimen, reg_dets in dup.groupby('regimen'):
    print(regimen)
    for lab, d in reg_dets.to_dict().items():
        if len(set(d.values())) != 1:
            print('\t', lab, d)

# should these variant definitions have the same cyclesig?

distinct_sigs = sig_vars[~sig_vars.component.str.contains('surg|brachy|radiotherapy', case=False)][['regimen', 'variant', 'phase', 'portion', 'cyclesigs', 'timing', 'branch']].drop_duplicates()
distinct_sigs[distinct_sigs[['regimen', 'variant', 'phase', 'portion', 'timing', 'branch']].duplicated(keep=False)][['regimen', 'variant']].drop_duplicates()

sig_vars[(sig_vars.regimen=='TACE, then 5-FU') & (sig_vars.variant=='Variant #01')][['regimen', 'variant', 'phase', 'portion', 'cyclesigs', 'timing', 'branch', 'component']]


# components in the variant file that aren't in the drug class - need to revisit to pull in procedures
sig_vars_components[sig_vars_components.concept_code.isna()]

BCG vaccine monotherapy
	 portion {1249: '-', 1250: '-', 1251: 'induction', 1252: 'induction', 1253: '-', 1254: '-'}
	 timing_sequence {1249: '1,8,15', 1250: '1,8,15', 1251: '1,8,15,22,29,36', 1252: '1,8,15,22,29,36', 1253: '1,8,15,22,29,36', 1254: '1,8,15,22,29,36'}
	 timing {1249: 'Days 1, 8, 15', 1250: 'Days 1, 8, 15', 1251: 'Days 1, 8, 15, 22, 29, 36', 1252: 'Days 1, 8, 15, 22, 29, 36', 1253: 'Days 1, 8, 15, 22, 29, 36', 1254: 'Days 1, 8, 15, 22, 29, 36'}
	 cyclesigs {1249: '6-week course, then proceed to maintenance therapy', 1250: '6-week course, then proceed to maintenance therapy', 1251: '6-week course, then proceed to maintenance therapy', 1252: '6-week course, then proceed to maintenance therapy', 1253: '6-week course', 1254: '6-week course'}
	 doseMinNum {1249: '0.5', 1250: '50', 1251: '0.5', 1252: '50', 1253: '0.5', 1254: '50'}
	 doseMaxNum {1249: '0.5', 1250: '50', 1251: '0.5', 1252: '50', 1253: '0.5', 1254: '50'}
	 doseUnit_cui {1249: nan, 1250: nan, 1251: nan, 1252: nan,

,study,regimen,regimen_cui,phase,portion,component,component_cui,component_role,cycle_length_lb,cycle_length_ub,...,seq.rel.what,tail,variant,variant_cui,temp,date_added,concept_name,concept_code,regimen_part_id,sig_id
0,MDACC ID01-233,"(90)YFC, then allo HSCT",19726,Consolidation,-,Allogeneic stem cells,38881,primary systemic,1,1,...,NaN,-,Variant #01,129496,NaN,D-2023-09-19,NaN,NaN,0,0
1,MDACC ID01-233,"(90)YFC, then allo HSCT",19726,Consolidation,-,Cyclophosphamide,122,primary systemic,1,1,...,NaN,-,Variant #01,129496,NaN,D-2023-09-19,NaN,NaN,0,1
2,MDACC ID01-233,"(90)YFC, then allo HSCT",19726,Consolidation,-,Fludarabine,224,primary systemic,1,1,...,NaN,-,Variant #01,129496,NaN,D-2023-09-19,NaN,NaN,0,2
5,MDACC ID01-233,"(90)YFC, then allo HSCT",19726,Consolidation,-,Rituximab,446,primary systemic,1,1,...,NaN,-,Variant #01,129496,NaN,D-2023-09-19,NaN,NaN,0,5
3,MDACC ID01-233,"(90)YFC, then allo HSCT",19726,Consolidation,-,Ibritumomab tiuxetan,262,primary systemic,1,1,...,NaN,-,Variant #01,129496,NaN,D-2023-09-19,NaN,NaN,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19874,Viviani et al. 1999,VEBEP,22971,NaN,-,Prednisolone,417,primary systemic,NaN,NaN,...,NaN,-,Variant #01,151027,NaN,D-2025-03-25,NaN,NaN,0,4
19994,MOTION,Vimseltinib monotherapy,144680,NaN,-,Vimseltinib,150561,primary systemic,7,7,...,NaN,-,Variant #01,151028,NaN,D-2025-03-25,NaN,NaN,0,0
20097,TROPION-Breast01,Vinorelbine monotherapy,6944,NaN,-,Vinorelbine,540,primary systemic,21,21,...,NaN,-,Variant #22,151029,look at this,D-2025-03-25,NaN,NaN,0,0
20400,Nesbit et al. 1981|CALGB 7611|Total Therapy XV,WBRT,10813,NaN,-,External beam radiotherapy,1608,locoregional,1,1,...,NaN,-,Variant #01,151030,NaN,D-2025-03-25,NaN,NaN,0,0


In [192]:

distinct_sigs = sig_vars[~sig_vars.component.str.contains('surg|brachy|radiotherapy', case=False)][['regimen', 'variant', 'portion', 'cyclesigs', 'timing', 'branch']].drop_duplicates()
distinct_sigs[distinct_sigs[['regimen', 'variant', 'portion', 'timing', 'branch']].duplicated(keep=False)][['regimen', 'variant']].drop_duplicates()


,regimen,variant
635,ADT and Darolutamide,Variant #01
660,"ADT, Enzalutamide, Talazoparib",Variant #01
2506,CapeOx and Pembrolizumab,Variant #01
3122,Carboplatin and Paclitaxel (CP) and Nivolumab,Variant #05
3126,Carboplatin and Paclitaxel (CP) and Nivolumab,Variant #06
3130,Carboplatin and Paclitaxel (CP) and Nivolumab,Variant #07
3134,Carboplatin and Paclitaxel (CP) and Nivolumab,Variant #08
3195,Carboplatin and Paclitaxel (CP) and Tislelizumab,Variant #02
3498,"Carboplatin, Pemetrexed, Tislelizumab",Variant #02
3855,Chlorambucil and Prednisolone,Variant #01


In [ ]:
ref[ref.concept_code.isna()].condition.value_counts()